In [15]:
#@title PY01 - Segmentación y Perfilamiento de Clientes en Retail
%matplotlib inline
#%load_ext google.colab.data_table
import pandas as pd
import numpy as np
import scipy.stats as stat
from datetime import datetime, time, timedelta
#from google.colab import data_table
import ipywidgets as wd
from IPython.display import display, clear_output
from io import StringIO, BytesIO
from IPython.utils import io
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import warnings
import math
import os
import sys
with io.capture_output() as captured:
    !pip install psycopg2
    import psycopg2
    !pip install pyod
    !pip install plotly --upgrade
    !pip install pymongo
import pymongo
from pyod.models.abod import ABOD
import plotly.express as px
import plotly.graph_objects as go
import pickle
warnings.filterwarnings("ignore")

# Para que la salida al visualizar tenga 1000 filas
pd.set_option('display.max_rows', 1000)

def corregir_tipo_de_datos_dataframe(df):
    df['fecNacimiento'] = pd.to_datetime(df['fecNacimiento'], yearfirst = True, format = '%y-%m-%d', exact = False)
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['monto'] = pd.to_numeric(df['monto'])
    df['descuento'] = pd.to_numeric(df['descuento'])
    return df

def crear_dataframe_desde_sql():
    cadena_conexion = ('postgresql+psycopg2://{nom_usuario}:{password}@{host}:{port}/{baseDatos}')
    cadena_conexion = cadena_conexion.format(
        nom_usuario = 'py01user',
        password = '201315252',
        host = '5.189.129.12',
        port = 5432,
        baseDatos = 'py01db'
    )
    motor = create_engine(cadena_conexion)
    query = 'SELECT cliente, NULLIF("fecNacimiento", {}) AS "fecNacimiento", sexo, "fechaVenta", monto, descuento FROM "datasource"'.format("'1900-01-01'")
    registros = motor.execute(query).fetchall()
    df = pd.DataFrame(registros, columns = ['cliente','fecNacimiento','sexo','fecha','monto','descuento'])
    return corregir_tipo_de_datos_dataframe(df)

def crear_dataframe_desde_archivo_cargado():
    archivo_cargado_metadata = next(iter(btn_cargar_archivo.value.items()))[1]['metadata']
    archivo_cargado_tipo = archivo_cargado_metadata['type']
    archivo_cargado_extension = os.path.splitext(archivo_cargado_metadata['name'])[1]
 
    df = None
    if archivo_cargado_extension == '.csv':
        try:
            df = pd.read_csv(StringIO(str(btn_cargar_archivo.data[0], 'utf-8')), na_values = '1900-01-01')
        except:
            df = pd.read_csv(StringIO(str(btn_cargar_archivo.data[0], '1252')), na_values = '1900-01-01')
    elif archivo_cargado_extension == '.xlsx' or archivo_cargado_tipo == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
        df = pd.read_excel(BytesIO(btn_cargar_archivo.data[0]), na_values = '1900-01-01')

    return corregir_tipo_de_datos_dataframe(df)

def validar_extension_archivo_cargado():
    archivo_cargado_nombre = next(iter(btn_cargar_archivo.value.keys()))
    archivo_cargado_extension = os.path.splitext(archivo_cargado_nombre)[1]
    if archivo_cargado_extension in ('.csv', '.xlsx'):
        return True
    else:
        print('Intenta nuevamente con un archivo csv o excel.')
        return False

df = None
def aplicar_correcciones_cliente():
    # Nada que corregir por el momento
    return True

def aplicar_correcciones_fecNacimiento():
    global df
    # Validamos que haya un solo fecNacimiento por cliente
    nacimiento = df.groupby('cliente')['fecNacimiento'].nunique()
    cantidad_clientes_multiples_fecNacimiento = sum(nacimiento > 1)
    if cantidad_clientes_multiples_fecNacimiento > 0:
        print('Hay {} clientes con más de una fecha de nacimiento.'.format(cantidad_clientes_multiples_fecNacimiento))
        df = None
        return False

    # Eliminamos los casos de año 13
    fecha_inicio = datetime(2013,1,1)
    fecha_fin = datetime(2020,12,1)
    condicion = df['fecNacimiento'].between(fecha_inicio, fecha_fin)
    df.drop(df.loc[condicion].index, axis = 0, inplace = True)
    # Estandarizar fechas
    df.loc[df['fecNacimiento'] > df['fecha'], 'fecNacimiento'] = [d.replace(year = d.year - 100) for d in df.loc[df['fecNacimiento'] > df['fecha']]['fecNacimiento']]
    # Convertir a NaN las fechas de 1900
    #df.loc[df['fecNacimiento'] == datetime(1900,1,1).date(), 'fecNacimiento'] = np.nan
    return True

def aplicar_correcciones_sexo():
    global df
    # Validamos que haya un solo sexo por cliente
    sexo = df.groupby('cliente')['sexo'].nunique()
    cantidad_clientes_multiples_sexo = sum(sexo > 1)
    if cantidad_clientes_multiples_sexo > 0:
        print('Hay {} clientes con más de un sexo.'.format(cantidad_clientes_multiples_sexo))
        df = None
        return False
    # Estandarizamos sexos
    df.loc[pd.isnull(df['sexo']) == False, 'sexo'] = df['sexo'].map(str.capitalize, na_action = 'ignore')
    return True
 
def aplicar_correcciones_fecha():
    # Nada que corregir por el momento
    return True
 
def aplicar_correcciones_monto():
    global df
    # Validamos que el monto sea mayor a 0
    montos_menor_cero = (df['monto'] < 0)
    cantidad_montos_menor_cero = len(df[montos_menor_cero])
    if cantidad_montos_menor_cero > 0:
        print('Hay {} compras con monto menor a 0.'.format(cantidad_montos_menor_cero))
        df = None
        return False
    # Nada que corregir por el momento
    return True
 
def aplicar_correcciones_descuento():
    global df
    # Validamos que el descuento sea mayor a 0
    descuentos_menor_cero = (df['monto'] < 0)
    cantidad_descuentos_menor_cero = len(df[descuentos_menor_cero])
    if cantidad_descuentos_menor_cero > 0:
        print('Hay {} compras con descuento menor a 0.'.format(cantidad_descuentos_menor_cero))
        df = None
        return False
    # Nada que corregir por el momento
    return True
 
def mostrar_mensaje_error_validacion_variable(nombre_variable):
    print('Corrija el error en la columna {} e intente nuevamente.'.format(nombre_variable))
 
def aplicar_correccion_dataframe_archivo_cargado():
    if not aplicar_correcciones_cliente(): mostrar_mensaje_error_validacion_variable('cliente')
    if not aplicar_correcciones_fecNacimiento(): mostrar_mensaje_error_validacion_variable('fecNacimiento')
    if not aplicar_correcciones_sexo(): mostrar_mensaje_error_validacion_variable('sexo')
    if not aplicar_correcciones_fecha(): mostrar_mensaje_error_validacion_variable('fecha')
    if not aplicar_correcciones_monto(): mostrar_mensaje_error_validacion_variable('monto')
    if not aplicar_correcciones_descuento(): mostrar_mensaje_error_validacion_variable('descuento')

def agregar_columna_hora_en_segundos():
    global df
    df['hora_en_segundos'] = df['fecha'].map(lambda x: x.hour * 3600 + x.minute * 60 + x.second)

def agregar_columnas():
    agregar_columna_hora_en_segundos()
    # agregar_columna_...()
    # agregar_columna_...()

def procesar_archivo(fuente):
    global df
    if fuente == 'SQL':
        df = crear_dataframe_desde_sql()
    elif fuente == 'Archivo':
        if validar_extension_archivo_cargado():
            df = crear_dataframe_desde_archivo_cargado()
    
    aplicar_correccion_dataframe_archivo_cargado()
    agregar_columnas()
    with out_mensaje_procesar_datos:
        clear_output()
        print('La data ha sido procesada correctamente.')

q_clientes = 2
df_indicadores_clientes = []
 
def obtener_indicadores_por_cliente():
    global df
    df_indicadores_clientes = df[df['cliente'] != 'SID1'][['cliente', 'monto']].groupby(['cliente']).sum()
    df_indicadores_clientes['cantidad'] = df[df['cliente'] != 'SID1'][['cliente']].groupby(['cliente']).size()
    df_indicadores_clientes.reset_index(inplace = True)
    df_indicadores_clientes.sort_values(by=['cantidad', 'monto'], ascending = False, inplace = True)
    return df_indicadores_clientes['cliente'].tolist()
 
def actualizar_variables_df():
    global q_clientes
    global df_indicadores_clientes
    q_clientes = df['cliente'].nunique() - 1 # Quitamos el SID1
    df_indicadores_clientes = obtener_indicadores_por_cliente()

def insertar_nuevo_registro(cliente, fecNacimiento, sexo, fecha, monto, descuento, hora_en_segundos):
    new_row = pd.DataFrame([[cliente, fecNacimiento, sexo, fecha, monto, descuento, hora_en_segundos]], columns = ['cliente','fecNacimiento','sexo','fecha','monto','descuento','hora_en_segundos'])
    return pd.concat([df, new_row], ignore_index = True)

def dividir_dataset_outliers_boxplot(base, nombre_columna, devolver_inliers):
    q1 = base[nombre_columna].quantile(0.25)
    q3 = base[nombre_columna].quantile(0.75)
    iqr = q3-q1
    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr
    condicion_outliers = ((base[nombre_columna] < lim_inf) | (base[nombre_columna] > lim_sup))
    if devolver_inliers:
        return (base[condicion_outliers], base[condicion_outliers == False])
    else:
        return base[condicion_outliers]

conf = 0.95
def dividir_dataset_outliers_mahalanobis(base, nombres_columnas, devolver_inliers):
    global conf
    data = np.array(base[nombres_columnas])
    data = np.transpose(data)
    avg = np.mean(data, axis = 1)
    cov_data = np.cov(data)
    inv_cov = np.linalg.inv(cov_data)
    diff = []
    for i in range(len(data)):
        diff.append([x_i - avg[i] for x_i in data[i]])
    diff_t = np.transpose(diff)
    distancias = []
    for i in range(len(diff_t)):
        distancias.append(np.dot(np.dot((diff_t[i]), inv_cov),np.transpose(diff_t[i])))
    critical_value = stat.chi2.ppf(q = conf, df = len(data))
    condicion_outliers = np.array([True if (y > critical_value) else False for y in distancias])
    if devolver_inliers:
        return (base[condicion_outliers], base[~condicion_outliers])
    else:
        return base[condicion_outliers]

#def norm(lista): # solo lo usa dividir_dataset_outliers_abod()
#    nrow, ncol = lista.shape
#    norm_matrix_AB = tuple([] for i in range(nrow))
#    norm_matrix_AC = tuple([] for i in range(nrow))
#
#    for i in range(nrow):
#        j = 0
#        for j in range(i+1):
#            if i == j:
#                norm_matrix_AB[i].append(0.0)
#                norm_matrix_AC[i].append(0.0)
#                continue
#            else:
#                dist2vec = lista[j] - lista[i]
#                #AB
#                norm_AB = math.sqrt(sum((k*k for k in dist2vec)))
#                norm_matrix_AB[i].append(norm_AB)
#                #AC
#                norm_AC = math.sqrt(sum((k*k for k in dist2vec)) + 0.01)
#                norm_matrix_AC[i].append(norm_AC)
#
#    return (norm_matrix_AB, norm_matrix_AC)

#def dividir_dataset_outliers_abod(base, nombres_columnas):
#    points = base[nombres_columnas].to_numpy()
#    norm_matrix_AB, norm_matrix_AC = norm(points)
#    nrow, ncol = points.shape
#    abod_list = []
#
#    for a in range(nrow):
#        angle_list = []
#        A = points[a]
#
#        for b in range(nrow):
#            if a == b: continue
#
#            B = points[b]
#            for c in range(b + 1, nrow):
#                if a == c: continue
#                   
#                C = points[c]
#                # producto escalar entre dos vectores
#                AB = B - A
#                AC = C - A
#                scalar_product = np.dot(AB, AC)
#                # norma AB de matriz
#                norm_AB = norm_matrix_AB[a][b] if a > b else norm_matrix_AB[b][a]
#                # norm AC de matriz
#                norm_AC = norm_matrix_AC[a][c] if a > c else norm_matrix_AC[c][a]
#                # estimación de ángulo
#                try:
#                    cos_AB_AC = scalar_product / (norm_AB * norm_AC)
#                    angle_AB_AC = math.acos(cos_AB_AC)
#                    factor_AB_AC = round(angle_AB_AC / (norm_AB * norm_AC), 2)
#                except ZeroDivisionError:
#                    sys.exit("Error! División por cero")
#
#                angle_list.append(factor_AB_AC)
#
#        abod = np.var(angle_list, ddof = 1)
#        abod_list.append(abod)
#
#    top_outliers = round(0.04 * nrow, 0)
#    base = base.assign(factor_abod = abod_list).sort_values('factor_abod').drop('factor_abod', axis = 1)
#
#    return (base.loc[:top_outliers], base.loc[top_outliers:])

def dividir_dataset_outliers_abod(base, nombres_columnas, devolver_inliers):
    clf = ABOD(method = 'fast', contamination = 0.05)
    clf.fit(base[nombres_columnas])
    predictions = clf.predict(base[nombres_columnas])
    condicion_outliers = predictions.astype(bool)
    if devolver_inliers:
        return (base[condicion_outliers], base[~condicion_outliers])
    else:
        return base[condicion_outliers]

def dividir_dataset_outliers(base, nombres_columnas, metodo, devolver_inliers):
    if metodo == 'Boxplot':
        return dividir_dataset_outliers_boxplot(base, nombres_columnas[0], devolver_inliers)
    elif metodo == 'Mahalanobis':
        return dividir_dataset_outliers_mahalanobis(base, nombres_columnas, devolver_inliers)
    elif metodo == 'ABOD':
        return dividir_dataset_outliers_abod(base, nombres_columnas, devolver_inliers)

#df_ordenado_criterio_1 = None
#df_ordenado_criterio_2 = None
#df_ordenado_criterio_3 = None
#df_ordenado_criterio_4 = None
#df_ordenado_criterio_5 = None

def ordenar_por_criterio_1(): # Cliente con compras frecuentes
    #global df_ordenado_criterio_1
    df_ordenado_criterio_1 = df[df['cliente'] != 'SID1'][['cliente', 'monto']].groupby(['cliente']).sum()
    df_ordenado_criterio_1['cantidad'] = df[df['cliente'] != 'SID1'][['cliente']].groupby(['cliente']).size()
    df_ordenado_criterio_1.reset_index(inplace = True)
    df_ordenado_criterio_1.sort_values(by=['cantidad', 'monto'], ascending = False, inplace = True)
    return df_ordenado_criterio_1['cliente'].tolist()

def ordenar_por_criterio_2(): # Cliente con compras de monto alto
    #global df_ordenado_criterio_2
    df_ordenado_criterio_2 = df[df['cliente'] != 'SID1'][['cliente', 'monto']].groupby(['cliente']).sum()
    df_ordenado_criterio_2['cantidad'] = df[df['cliente'] != 'SID1'][['cliente']].groupby(['cliente']).size()
    df_ordenado_criterio_2.reset_index(inplace = True)
    df_ordenado_criterio_2.sort_values(by=['monto', 'cantidad'], ascending = False, inplace = True)
    return df_ordenado_criterio_2['cliente'].tolist()

def ordenar_por_criterio_3():
    pass

def ordenar_por_criterio_4():
    pass

def ordenar_por_criterio_5():
    pass

def obtener_n_clientes(indicadores_clientes, n, flag_mejores):
    if flag_mejores:
        return indicadores_clientes[:n]
    else:
        return indicadores_clientes[-n:][::-1]

def obtener_curvas_aprendizaje():
    pickle_in = open('pickles/validacion.pkl', 'rb')
    curvas_aprendizaje = pickle.load(pickle_in)
    pickle_in.close()
    return curvas_aprendizaje

def obtener_scores_clasificadores():
    pickle_in = open('pickles/clasificadores_v2.pkl', 'rb')
    clasificadores = pickle.load(pickle_in)
    pickle_in.close()
    return clasificadores

def crear_dataframe_desde_mongodb():
    cadena_conexion = ('mongodb://{nom_usuario}:{password}@{host}:{port}')
    cadena_conexion = cadena_conexion.format(
        nom_usuario = 'PY01_c02',
        password = 'P4rd83XkXrTz',
        host = '5.189.129.12',
        port = 27017
    )
    client = pymongo.MongoClient(cadena_conexion)
    db = client.PY01
    header = [*db.PY01.find_one().keys()]
    lista_dataset = list()
    for doc in db.PY01.find():
        lista_dataset.append([*doc.values()])
 
    df_modelo = pd.DataFrame(data = lista_dataset, columns = header)
    return df_modelo
    
gbx_tabla_etiquetado = None
tab_etiquetado = None
flag_tab_etiquetado_mostrado = False
def mostrar_widgets_visualizacion(): 
    actualizar_variables_df()
    indicadores_clientes = ordenar_por_criterio_1()
    #criterios_relevancia = ['Frecuencia de compra','Monto de compra', 'Criterio 3', 'Criterio 4', 'Criterio 5']
    criterios_relevancia = ['Frecuencia de compra','Monto de compra']
 
    # Widgets del tab_children_1
    html_descripcion_tab_1 = wd.HTML('''<h2>Clientes más relevantes</h2><br>En esta sección puedes escoger un criterio de relevancia y obtener un top N de clientes más relevantes según tal criterio, como también ver a detalle un cliente de la lista obtenida.''')
    dpw_criterio_mas_relevantes = wd.Dropdown(options = criterios_relevancia, value = criterios_relevancia[0])
    def handle_dwp_criterio_mas_relevantes_change(change):
        global indicadores_clientes
        if change.new == criterios_relevancia[0]:
            indicadores_clientes = ordenar_por_criterio_1()
        elif change.new == criterios_relevancia[1]:
            indicadores_clientes = ordenar_por_criterio_2()
        elif change.new == criterios_relevancia[2]:
            #indicadores_clientes = ordenar_por_criterio_3()
            return
        elif change.new == criterios_relevancia[3]:
            #indicadores_clientes = ordenar_por_criterio_4()
            return
        elif change.new == criterios_relevancia[4]:
            #indicadores_clientes = ordenar_por_criterio_5()
            return
        slt_seleccionar_cliente_mas_relevantes.options = obtener_n_clientes(indicadores_clientes, int_n_clientes_mas_relevantes.value, True)
    dpw_criterio_mas_relevantes.observe(handle_dwp_criterio_mas_relevantes_change, names = 'value')

    int_n_clientes_mas_relevantes = wd.BoundedIntText(value = 5, min = 1, max = q_clientes, step = 1, disabled = False, large = 1)    

    lista_mejores_clientes = obtener_n_clientes(indicadores_clientes, int_n_clientes_mas_relevantes.get_interact_value(), True)
    slt_seleccionar_cliente_mas_relevantes = wd.Select(options = lista_mejores_clientes, value = None, rows = 5, disabled = False)
 
    def handle_int_n_mas_relevantes_change(change):
        slt_seleccionar_cliente_mas_relevantes.options = obtener_n_clientes(indicadores_clientes, change.new, True)
    int_n_clientes_mas_relevantes.observe(handle_int_n_mas_relevantes_change, names = 'value')
    
    def mostrar_graficos(valor):
        data1 = df[df['cliente'] == valor]['fecha'].dt.strftime('%Y-%m').value_counts().sort_index()
        data2 = df[df['cliente'] == valor]['fecha'].dt.strftime('%u').value_counts().sort_index().rename(index = {'1':'L','2':'Ma','3':'Mi','4':'J','5':'V','6':'S','7':'D'})
        data3 = df[df['cliente'] == valor][['fecha', 'monto']]
        data3['periodo'] = data3['fecha'].dt.strftime('%Y-%m')
        data3 = data3.groupby(['periodo']).sum().reset_index().set_index('periodo')['monto'].sort_index()
        data4 = df[df['cliente'] == valor][['fecha', 'monto']]
        data4['dia_semana'] = data4['fecha'].dt.strftime('%u')
        data4 = data4.groupby(['dia_semana']).sum().reset_index().set_index('dia_semana')['monto'].sort_index().rename(index = {'1':'L','2':'Ma','3':'Mi','4':'J','5':'V','6':'S','7':'D'})
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)
        fig.set_size_inches(12, 10)
        fig.suptitle('Cliente {}'.format(valor), fontsize = 14)
 
        ax1.bar(np.arange(data1.size), data1.values)
        ax1.set_title('Meses de consumo')
        ax1.set_xticks(np.arange(len(data1)))
        ax1.set_xticklabels(data1.keys(), rotation = 60, fontdict = None, minor = False)
        #ax1.set_xlabel('Meses')
        ax1.set_ylabel('Frecuencia de compra')
 
        ax2.bar(np.arange(data2.size), data2.values, color = 'tab:green')
        ax2.set_title('Días de consumo')
        ax2.set_xticks(np.arange(len(data2)))
        ax2.set_xticklabels(data2.keys(), fontdict = None, minor = False)
        #ax2.set_xlabel('Días de la semana')
        #ax2.set_ylabel('Cantidad de compras')
 
        ax3.bar(np.arange(data3.size), data3.values)
        #ax3.set_title('Meses de consumo')
        ax3.set_xticks(np.arange(len(data3)))
        ax3.set_xticklabels(data3.keys(), rotation = 60, fontdict = None, minor = False)
        ax3.set_xlabel('Meses')
        ax3.set_ylabel('Monto de compra')
 
        ax4.bar(np.arange(data4.size), data4.values, color = 'tab:green')
        #ax4.set_title('Días de consumo')
        #ax4.set_xticks(np.arange(len(data4)))
        ax4.set_xticklabels(data4.keys(), fontdict = None, minor = False)
        ax4.set_xlabel('Días de la semana')
        #ax4.set_ylabel('Cantidad de compras')
 
        plt.show()

    graficos_mas_relevantes = wd.Output()
    def handle_select_cliente_mas_relevantes_change(change):
        with graficos_mas_relevantes:
            clear_output()
            mostrar_graficos(change.new)
    slt_seleccionar_cliente_mas_relevantes.observe(handle_select_cliente_mas_relevantes_change, names = 'value')

    box_controles_tab_1 = wd.VBox(children = [wd.Label(value = 'Seleccione un criterio de relevancia:'),
                                              dpw_criterio_mas_relevantes,
                                              wd.Label(value = 'Seleccione la cantidad de clientes que desea obtener:'),
                                              int_n_clientes_mas_relevantes,
                                              wd.Label(value = 'Seleccione un cliente:'),
                                              slt_seleccionar_cliente_mas_relevantes])
    tab_children_1 = wd.VBox(children = [html_descripcion_tab_1, wd.HBox(children = [box_controles_tab_1, graficos_mas_relevantes])])
 
    # Widgets del tab_children_2
    html_descripcion_tab_2 = wd.HTML('''<h2>Clientes menos relevantes</h2><br>En esta sección puedes escoger un criterio de relevancia y obtener un top N de clientes menos relevantes según tal criterio, como también ver a detalle un cliente de la lista obtenida.''')
    dpw_criterio_menos_relevantes = wd.Dropdown(options = criterios_relevancia, value = criterios_relevancia[0])
    def handle_dwp_criterio_menos_relevantes_change(change):
        global indicadores_clientes
        if change.new == criterios_relevancia[0]:
            indicadores_clientes = ordenar_por_criterio_1()
        elif change.new == criterios_relevancia[1]:
            indicadores_clientes = ordenar_por_criterio_2()
        elif change.new == criterios_relevancia[2]:
            #indicadores_clientes = ordenar_por_criterio_3()
            return
        elif change.new == criterios_relevancia[3]:
            #indicadores_clientes = ordenar_por_criterio_4()
            return
        elif change.new == criterios_relevancia[4]:
            #indicadores_clientes = ordenar_por_criterio_5()
            return
        slt_seleccionar_cliente_menos_relevantes.options = obtener_n_clientes(indicadores_clientes, int_n_clientes_menos_relevantes.value, False)
    dpw_criterio_menos_relevantes.observe(handle_dwp_criterio_menos_relevantes_change, names = 'value')

    int_n_clientes_menos_relevantes = wd.BoundedIntText(value = 5, min = 1, max = q_clientes, step = 1, disabled = False, large = 1)
    
    lista_peores_clientes = obtener_n_clientes(indicadores_clientes, int_n_clientes_menos_relevantes.get_interact_value(), False)
    slt_seleccionar_cliente_menos_relevantes = wd.Select(options = lista_peores_clientes, value = None, rows = 5, disabled = False)
 
    def handle_int_n_menos_relevantes_change(change):
        slt_seleccionar_cliente_menos_relevantes.options = obtener_n_clientes(indicadores_clientes, change.new, False)    
    int_n_clientes_menos_relevantes.observe(handle_int_n_menos_relevantes_change, names = 'value')
    
    graficos_menos_relevantes = wd.Output()
    def handle_select_cliente_menos_relevantes_change(change):
        with graficos_menos_relevantes:
            clear_output()
            mostrar_graficos(change.new)
    slt_seleccionar_cliente_menos_relevantes.observe(handle_select_cliente_menos_relevantes_change, names = 'value')
 
    box_controles_tab_2 = wd.VBox(children = [wd.Label(value = 'Seleccione un criterio de relevancia:'),
                                              dpw_criterio_menos_relevantes,
                                              wd.Label(value = 'Seleccione la cantidad de clientes que desea obtener:'),
                                              int_n_clientes_menos_relevantes,
                                              wd.Label(value = 'Seleccione un cliente:'),
                                              slt_seleccionar_cliente_menos_relevantes])
    tab_children_2 = wd.VBox(children = [html_descripcion_tab_2, wd.HBox(children = [box_controles_tab_2, graficos_menos_relevantes])])
    
    # Widgets del tab_children_3
    html_descripcion_tab_3 = wd.HTML('<h2>Ingreso de un nuevo registro</h2><br>En esta sección puedes ingresar un registro adicional a los del dataset que subiste.')
    #lbl_new_cliente = wd.Label('ID del cliente:')
    #box_int_new_cliente = wd.BoundedIntText(value = 1, min = 1, max = 100000, step = 1)
    #lbl_new_fecNacimiento = wd.Label('Fecha de nacimiento del cliente:')
    #box_dte_new_fecNacimiento = wd.DatePicker()
    #def handle_box_dte_new_fecNacimiento_update(change):
    #    if change.new is not None and change.new > datetime.now().date():
    #        box_txt_new_fecNacimiento.value = None
    #        with out_avisos_datos:
    #            clear_output()
    #            print('Ingrese una fecha válida.')
    #box_dte_new_fecNacimiento.observe(handle_box_dte_new_fecNacimiento_update, names = 'value')
    #lbl_new_sexo = wd.Label('Sexo del cliente:')
    #dpw_new_sexo = wd.Dropdown(options = ['Masculino','Femenino'], value = None)
    lbl_new_fecha = wd.Label('Fecha y hora de compra:')
    time_layout = wd.Layout(flex='1 1 auto', width = '25px')
    box_dte_new_fecha = wd.DatePicker(layout = wd.Layout(flex='1 1 auto', width = '130px'))
    def handle_box_dte_new_fecha_update(change):
        if change.new is not None and change.new > datetime.now().date():
            box_dte_new_fecha.value = None
            with out_avisos_datos:
                clear_output()
                print('Ingrese una fecha válida.')
    box_dte_new_fecha.observe(handle_box_dte_new_fecha_update, names = 'value')
    box_int_new_hora = wd.BoundedIntText(value = 0, min = 0, max = 23, step = 1, layout = time_layout)
    box_int_new_minuto = wd.BoundedIntText(value = 0, min = 0, max = 59, step = 1, layout = time_layout)
    box_int_new_segundo = wd.BoundedIntText(value = 0, min = 0, max = 59, step = 1, layout = time_layout)
    lbl_new_fecha_time = wd.Label(':')
    hbx_dtt_new_fecha = wd.HBox(children = [box_dte_new_fecha, box_int_new_hora, box_int_new_minuto, box_int_new_segundo], layout = wd.Layout(display = 'flex', flex_flow = 'row', align_items = 'stretch'))
    lbl_new_monto = wd.Label('Monto de la compra:')
    box_flt_new_monto = wd.BoundedFloatText(value = 0, min = 0, max = 100000, step = 5)
    lbl_new_descuento = wd.Label('Descuento de la compra:')
    box_flt_new_descuento = wd.BoundedFloatText(value = 0, min = 0, max = 100000, step = 5)
    btn_ingresar_datos = wd.Button(description = 'Ingresar registro')

    def limpiar_formulario():
        #box_int_new_cliente.value = 1
        #box_dte_new_fecNacimiento.value = None
        #dpw_new_sexo.value = None
        box_dte_new_fecha.value = None
        box_int_new_hora.value = 0
        box_int_new_minuto.value = 0
        box_int_new_segundo.value = 0
        box_flt_new_monto.value = 0
        box_flt_new_descuento.value = 0

    def esta_incluido(df, fila):
        if len(df[(df['monto'] == fila[0]) & (df['descuento'] == fila[1]) & (df['hora_en_segundos'] == fila[2])]) > 0:
            return True
        else:
            return False

    def on_click_btn_ingresar_datos(b):
        if box_dte_new_fecha.value is None:
            box_dte_new_fecha.value = None
            with out_avisos_datos:
                clear_output()
                print('Ingrese una fecha válida.')
            return
        #cliente = 'SID' + str(box_int_new_cliente.value)
        #fecNacimiento = box_dte_new_fecNacimiento.value
        #sexo = dpw_new_sexo.value
        fecha = datetime.combine(box_dte_new_fecha.value,
                                time(hour = box_int_new_hora.value,
                                    minute = box_int_new_minuto.value,
                                    second = box_int_new_segundo.value))
        monto = box_flt_new_monto.value
        descuento = box_flt_new_descuento.value
        hora_en_segundos = fecha.hour * 3600 + fecha.minute * 60 + fecha.second
        nuevo_df = insertar_nuevo_registro('SID1', datetime.now(), 'F', fecha, monto, descuento, hora_en_segundos)
        
        flag_outlier_boxplot_monto = esta_incluido(dividir_dataset_outliers(nuevo_df, ['monto'], 'Boxplot', False), (monto, descuento, hora_en_segundos))
        flag_outlier_boxplot_dscto = esta_incluido(dividir_dataset_outliers(nuevo_df, ['descuento'], 'Boxplot', False), (monto, descuento, hora_en_segundos))
        flag_outlier_boxplot_hora = esta_incluido(dividir_dataset_outliers(nuevo_df, ['hora_en_segundos'], 'Boxplot', False), (monto, descuento, hora_en_segundos))
        flag_outlier_mahalanobis_monto_dscto = esta_incluido(dividir_dataset_outliers(nuevo_df, ['monto','descuento'], 'Mahalanobis', False), (monto, descuento, hora_en_segundos))
        flag_outlier_mahalanobis_monto_hora = esta_incluido(dividir_dataset_outliers(nuevo_df, ['monto','hora_en_segundos'], 'Mahalanobis', False), (monto, descuento, hora_en_segundos))
        flag_outlier_mahalanobis_dscto_hora = esta_incluido(dividir_dataset_outliers(nuevo_df, ['descuento','hora_en_segundos'], 'Mahalanobis', False), (monto, descuento, hora_en_segundos))
        flag_outlier_abod_monto_dscto = esta_incluido(dividir_dataset_outliers(nuevo_df, ['monto','descuento'], 'ABOD', False), (monto, descuento, hora_en_segundos))
        flag_outlier_abod_monto_hora = esta_incluido(dividir_dataset_outliers(nuevo_df, ['monto','hora_en_segundos'], 'ABOD', False), (monto, descuento, hora_en_segundos))
        flag_outlier_abod_dscto_hora = esta_incluido(dividir_dataset_outliers(nuevo_df, ['descuento','hora_en_segundos'], 'ABOD', False), (monto, descuento, hora_en_segundos))
        #limpiar_formulario()
        
        dict_resultado_registro = {True : 'Outlier', False : 'Inlier'}
        with out_avisos_datos:
            clear_output()
            print('Con los datos ingresados, se han obtenido los siguientes resultados.')
            print('- Boxplot:')
            print('\t- Monto:', dict_resultado_registro[flag_outlier_boxplot_monto])
            print('\t- Descuento:', dict_resultado_registro[flag_outlier_boxplot_dscto])
            print('\t- Hora (fecVenta):', dict_resultado_registro[flag_outlier_boxplot_hora])
            print('- Mahalanobis:')
            print('\t- Monto y descuento:', dict_resultado_registro[flag_outlier_mahalanobis_monto_dscto])
            print('\t- Monto y hora (fecVenta):', dict_resultado_registro[flag_outlier_mahalanobis_monto_hora])
            print('\t- Descuento y hora (fecVenta):', dict_resultado_registro[flag_outlier_mahalanobis_dscto_hora])
            print('- ABOD:')
            print('\t- Monto y descuento:', dict_resultado_registro[flag_outlier_abod_monto_dscto])
            print('\t- Monto y hora (fecVenta):', dict_resultado_registro[flag_outlier_abod_monto_hora])
            print('\t- Descuento y hora (fecVenta):', dict_resultado_registro[flag_outlier_abod_dscto_hora])

    btn_ingresar_datos.on_click(on_click_btn_ingresar_datos)
    out_avisos_datos = wd.Output(layout = wd.Layout(margin = '10px 0 0 0'))

    grid_box_items = [
                      #lbl_new_cliente, box_int_new_cliente,
                      #lbl_new_fecNacimiento, box_dte_new_fecNacimiento,
                      #lbl_new_sexo, dpw_new_sexo,
                      lbl_new_fecha, hbx_dtt_new_fecha,
                      lbl_new_monto, box_flt_new_monto,
                      lbl_new_descuento, box_flt_new_descuento,
                      btn_ingresar_datos]
    tab_children_3 = wd.VBox(children = [html_descripcion_tab_3, wd.GridBox(children = grid_box_items, layout = wd.Layout(grid_template_columns = '200px 304px')), out_avisos_datos])
    
    # Widgets del tab_children_4
    html_descripcion_tab_4 = wd.HTML('<h2>Outliers a detalle</h2><br>En esta sección puedes observar en detalle los outliers de todos los clientes o de uno en particular, como también escoger el algoritmo más adecuado para la detección de estos.')
    lista_clientes_outliers = sorted(df['cliente'].unique(), key = lambda x: int(x[3:]))
    lista_clientes_outliers.insert(0, 'Todos')
    lbl_clientes_outliers = wd.Label('Seleccione un cliente:')
    dpw_clientes_outliers = wd.Dropdown(options = lista_clientes_outliers, value = 'Todos')

    lbl_confianza_mahalanobis = wd.Label('Indique el nivel de confianza (Mahalanobis):')
    box_flt_confianza_mahalanobis = wd.BoundedFloatText(value = 0.95, min = 0, max = 1, step = 0.01)
    def handle_box_flt_confianza_mahalanobis_update(change):
        global conf
        conf = change.new
    box_flt_confianza_mahalanobis.observe(handle_box_flt_confianza_mahalanobis_update, names = 'value')

    lbl_sliders = wd.Label('Seleccione el rango de valores a considerar para:')
    min_monto = min(df['monto'])
    max_monto = max(df['monto'])
    sld_monto = wd.IntRangeSlider(value = (min_monto, max_monto), min = min_monto, max = max_monto, step = 1, continuous_update = False, description = 'monto', orientation = 'horizontal', readout = True)

    min_dscto = min(df['descuento'])
    max_dscto = max(df['descuento'])
    sld_dscto = wd.IntRangeSlider(value = (min_dscto, max_dscto), min = min_dscto, max = max_dscto, step = 1, continuous_update = False, description = 'descuento', orientation = 'horizontal', readout = True)

    min_hora = 0
    max_hora = 60 * 24 - 1
    sld_hora = wd.IntRangeSlider(value = (min_hora, max_hora), min = min_hora, max = max_hora, step = 1, continuous_update = False, description = 'hora (mins)', orientation = 'horizontal', readout = True)
    lbl_slider_hora = wd.Label('00:00 - 23:59', layout = wd.Layout(display = 'flex', justify_content = 'flex-end'))
    def handle_sld_hora_update(change):
        lbl_slider_hora.value = '{} - {}'.format(datetime.fromtimestamp(change.new[0] * 60).strftime('%H:%M'), datetime.fromtimestamp(change.new[1] * 60).strftime('%H:%M'))
    sld_hora.observe(handle_sld_hora_update, names = 'value')

    def handle_dpw_clientes_outliers_update(change):
        global df
        if change.new == 'Todos':
            min_monto = min(df['monto'])
            max_monto = max(df['monto'])
            min_dscto = min(df['descuento'])
            max_dscto = max(df['descuento'])
            min_hora = 0
            max_hora = 60 * 24 - 1
        else:
            df_cliente = df[df['cliente'] == change.new]
            min_monto = min(df_cliente['monto'])
            max_monto = max(df_cliente['monto'])
            min_dscto = min(df_cliente['descuento'])
            max_dscto = max(df_cliente['descuento'])
            min_hora = math.ceil(min(df_cliente['hora_en_segundos'])) / 60.0
            max_hora = math.ceil(max(df_cliente['hora_en_segundos'])) / 60.0

        if min_monto > sld_monto.max:
            sld_monto.max = max_monto
            sld_monto.min = min_monto
        else:
            sld_monto.min = min_monto
            sld_monto.max = max_monto
        sld_monto.value = (min_monto, max_monto)

        if min_dscto > sld_dscto.max:
            sld_dscto.max = max_dscto
            sld_dscto.min = min_dscto
        else:
            sld_dscto.min = min_dscto
            sld_dscto.max = max_dscto
        sld_dscto.value = (min_dscto, max_dscto)

        if min_hora > sld_hora.max:
            sld_hora.max = max_hora
            sld_hora.min = min_hora
        else:
            sld_hora.min = min_hora
            sld_hora.max = max_hora
        sld_hora.value = (min_hora, max_hora)

    dpw_clientes_outliers.observe(handle_dpw_clientes_outliers_update, names = 'value')

    btn_refrescar_graficos_outliers = wd.Button(description = 'Graficar')

    out_boxplots = wd.Output()
    out_boxplots_outliers_monto = wd.Output()
    out_boxplots_outliers_dscto = wd.Output()
    out_boxplots_outliers_hora = wd.Output()
    out_mahalanobis = wd.Output()
    out_mahalanobis_outliers_monto_dscto = wd.Output()
    out_mahalanobis_outliers_monto_hora = wd.Output()
    out_mahalanobis_outliers_dscto_hora = wd.Output()
    out_abod = wd.Output()
    out_abod_outliers_monto_dscto = wd.Output()
    out_abod_outliers_monto_hora = wd.Output()
    out_abod_outliers_dscto_hora = wd.Output()

    acd_boxplots = wd.Accordion(children = [out_boxplots, out_boxplots_outliers_monto, out_boxplots_outliers_dscto, out_boxplots_outliers_hora], selected_index = None)
    acd_boxplots.set_title(0, 'Gráficos')
    acd_boxplots.set_title(1, 'Tabla de outliers de "monto"')
    acd_boxplots.set_title(2, 'Tabla de outliers de "descuento"')
    acd_boxplots.set_title(3, 'Tabla de outliers de la hora de "fecha"')
    acd_mahalanobis = wd.Accordion(children = [out_mahalanobis, out_mahalanobis_outliers_monto_dscto, out_mahalanobis_outliers_monto_hora, out_mahalanobis_outliers_dscto_hora], selected_index = None)
    acd_mahalanobis.set_title(0, 'Gráficos')
    acd_mahalanobis.set_title(1, 'Tabla de outliers de "monto" y "descuento"')
    acd_mahalanobis.set_title(2, 'Tabla de outliers de "monto" y la hora de "fecha"')
    acd_mahalanobis.set_title(3, 'Tabla de outliers de "descuento" y la hora de "fecha"')
    acd_abod = wd.Accordion(children = [out_abod, out_abod_outliers_monto_dscto, out_abod_outliers_monto_hora, out_abod_outliers_dscto_hora], selected_index = None)
    acd_abod.set_title(0, 'Gráficos')
    acd_abod.set_title(1, 'Tabla de outliers de "monto" y "descuento"')
    acd_abod.set_title(2, 'Tabla de outliers de "monto" y la hora de "fecha"')
    acd_abod.set_title(3, 'Tabla de outliers de "descuento" y la hora de "fecha"')

    out_graficos_tab_4 = wd.Output(layout = wd.Layout(margin = '15px 0 0 10px'))
    tab_graficos_tab_4 = wd.Tab(children = [acd_boxplots, acd_mahalanobis, acd_abod])
    tab_graficos_tab_4.set_title(0, 'Boxplot')
    tab_graficos_tab_4.set_title(1, 'Mahalanobis')
    tab_graficos_tab_4.set_title(2, 'ABOD')

    segundos_boxplot_ticks = [h * 3600 if h != 24 else h * 3600 - 1 for h in range(25)]
    hora_boxplot_ticks = [datetime.fromtimestamp(segundo).strftime('%H:%M:%S') for segundo in segundos_boxplot_ticks]

    def on_click_btn_refrescar_graficos_outliers(b):
        rango_monto = sld_monto.value
        rango_dscto = sld_dscto.value
        rango_hora = sld_hora.value
        cliente = dpw_clientes_outliers.value
        condicion_slider_monto = ((df['monto'] >= rango_monto[0]) & (df['monto'] <= rango_monto[1]))
        condicion_slider_dscto = ((df['descuento'] >= rango_dscto[0]) & (df['descuento'] <= rango_dscto[1]))
        condicion_slider_hora = ((df['hora_en_segundos'] >= rango_hora[0] * 60) & (df['hora_en_segundos'] <= rango_hora[1] * 60))
        condicion_slider_monto_dscto = condicion_slider_monto & condicion_slider_dscto
        condicion_slider_monto_hora = condicion_slider_monto & condicion_slider_hora
        condicion_slider_dscto_hora = condicion_slider_dscto & condicion_slider_hora
        if cliente != 'Todos':
            condicion_cliente = (df['cliente'] == cliente)
            condicion_slider_monto = (condicion_slider_monto & condicion_cliente)
            condicion_slider_dscto = (condicion_slider_dscto & condicion_cliente)
            condicion_slider_hora = (condicion_slider_hora & condicion_cliente)
            condicion_slider_monto_dscto = (condicion_slider_monto_dscto & condicion_cliente)
            condicion_slider_monto_hora = (condicion_slider_monto_hora & condicion_cliente)
            condicion_slider_dscto_hora = (condicion_slider_dscto_hora & condicion_cliente)
        df_boxplot_monto = dividir_dataset_outliers(df[condicion_slider_monto], ['monto'], 'Boxplot', False)
        df_boxplot_dscto = dividir_dataset_outliers(df[condicion_slider_dscto], ['descuento'], 'Boxplot', False)
        df_boxplot_hora = dividir_dataset_outliers(df[condicion_slider_hora], ['hora_en_segundos'], 'Boxplot', False)
        df_mahalanobis_monto_dscto = dividir_dataset_outliers(df[condicion_slider_monto_dscto], ['monto','descuento'], 'Mahalanobis', True)
        df_mahalanobis_monto_hora = dividir_dataset_outliers(df[condicion_slider_monto_hora], ['monto','hora_en_segundos'], 'Mahalanobis', True)
        df_mahalanobis_dscto_hora = dividir_dataset_outliers(df[condicion_slider_dscto_hora], ['descuento','hora_en_segundos'], 'Mahalanobis', True)
        df_abod_monto_dscto = dividir_dataset_outliers(df[condicion_slider_monto_dscto], ['monto','descuento'], 'ABOD', True)
        df_abod_monto_hora = dividir_dataset_outliers(df[condicion_slider_monto_hora], ['monto','hora_en_segundos'], 'ABOD', True)
        df_abod_dscto_hora = dividir_dataset_outliers(df[condicion_slider_dscto_hora], ['descuento','hora_en_segundos'], 'ABOD', True)
        #df_graficos_tab_4 = pd.melt(df[condicion_sliders][['monto','descuento','hora_en_segundos']])

        color_alert = 'rgba(219, 64, 82, 0.6)'
        color_no_alert = 'rgb(105, 105, 105)'
        with out_graficos_tab_4:
            print('Espere por favor...')
            clear_output(True)
            with out_boxplots:
                print('Espere por favor...')
                clear_output(True)
                fig1 = go.Figure()
                fig1.add_trace(go.Box(x = df[condicion_slider_monto]['monto'], orientation = 'h', name = '', boxpoints = 'outliers', marker_color = color_alert, line_color = color_no_alert))
                fig1.update_layout(title_text = 'Diagrama de cajas de la variable "monto" del cliente {} en soles'.format(cliente), autosize = False, width = 1000, height = 250)
                fig1.show()

                fig2 = go.Figure()
                fig2.add_trace(go.Box(x = df[condicion_slider_dscto]['descuento'], orientation = 'h', name = '', boxpoints = 'outliers', marker_color = color_alert, line_color = color_no_alert))
                fig2.update_layout(title_text = 'Diagrama de cajas de la variable "descuento" del cliente {} en soles'.format(cliente), autosize = False, width = 1000, height = 250)
                fig2.show()

                fig3 = go.Figure()
                fig3.add_trace(go.Box(x = df[condicion_slider_hora]['hora_en_segundos'], orientation = 'h', name = '', boxpoints = 'outliers', marker_color = color_alert, line_color = color_no_alert))
                fig3.update_layout(title_text = 'Diagrama de cajas de la hora (en mins) de la variable "fecVenta" del cliente {}'.format(cliente), autosize = False, width = 1000, height = 250)
                fig3.layout['xaxis1'].tickvals = segundos_boxplot_ticks
                fig3.layout['xaxis1'].ticktext = hora_boxplot_ticks
                fig3.show()
            with out_boxplots_outliers_monto:
                clear_output()
                display(df_boxplot_monto)
            with out_boxplots_outliers_dscto:
                clear_output()
                display(df_boxplot_dscto)
            with out_boxplots_outliers_hora:
                clear_output()
                display(df_boxplot_hora)
            with out_mahalanobis:
                print('Espere por favor...')
                clear_output(True)
                fig4 = go.Figure()
                fig4.add_trace(go.Scatter(name = 'outliers',mode = 'markers', x = df_mahalanobis_monto_dscto[0]['monto'], y = df_mahalanobis_monto_dscto[0]['descuento'], marker = dict(color = color_alert)))
                fig4.add_trace(go.Scatter(name = 'inliers',mode = 'markers', x = df_mahalanobis_monto_dscto[1]['monto'], y = df_mahalanobis_monto_dscto[1]['descuento'], marker = dict(color = color_no_alert)))
                fig4.update_layout(title_text = 'Gráfico de dispersión de la variable "monto" vs. "descuento" del cliente {}'.format(cliente), autosize = False, width = 1000, height = 400)
                fig4.layout['yaxis1'].title = 'descuento (soles)'
                fig4.layout['xaxis1'].title = 'monto (soles)'
                fig4.show()

                fig5 = go.Figure()
                fig5.add_trace(go.Scatter(name = 'outliers', mode = 'markers', x = df_mahalanobis_monto_hora[0]['hora_en_segundos'], y = df_mahalanobis_monto_hora[0]['monto'], marker = dict(color = color_alert)))
                fig5.add_trace(go.Scatter(name = 'inliers',mode = 'markers', x = df_mahalanobis_monto_hora[1]['hora_en_segundos'], y = df_mahalanobis_monto_hora[1]['monto'], marker = dict(color = color_no_alert)))
                fig5.update_layout(title_text = 'Gráfico de dispersión de la variable "monto" vs. hora de "fecVenta" del cliente {}'.format(cliente), autosize = False, width = 1000, height = 400)
                fig5.layout['yaxis1'].title = 'monto (soles)'
                fig5.layout['xaxis1'].title = 'fecVenta (hora)'
                fig5.layout['xaxis1'].tickvals = segundos_boxplot_ticks
                fig5.layout['xaxis1'].ticktext = hora_boxplot_ticks
                fig5.show()

                fig6 = go.Figure()
                fig6.add_trace(go.Scatter(name = 'outliers', mode = 'markers', x = df_mahalanobis_dscto_hora[0]['hora_en_segundos'], y = df_mahalanobis_dscto_hora[0]['descuento'], marker = dict(color = color_alert)))
                fig6.add_trace(go.Scatter(name = 'inliers', mode = 'markers', x = df_mahalanobis_dscto_hora[1]['hora_en_segundos'], y = df_mahalanobis_dscto_hora[1]['descuento'], marker = dict(color = color_no_alert)))
                fig6.update_layout(title_text = 'Gráfico de dispersión de la variable "descuento" vs. hora de "fecVenta" del cliente {}'.format(cliente), autosize = False, width = 1000, height = 400)
                fig6.layout['yaxis1'].title = 'descuento (soles)'
                fig6.layout['xaxis1'].title = 'fecVenta (hora)'
                fig6.layout['xaxis1'].tickvals = segundos_boxplot_ticks
                fig6.layout['xaxis1'].ticktext = hora_boxplot_ticks
                fig6.show()
            with out_mahalanobis_outliers_monto_dscto:
                clear_output()
                display(df_mahalanobis_monto_dscto[0])
            with out_mahalanobis_outliers_monto_hora:
                clear_output()
                display(df_mahalanobis_monto_hora[0])
            with out_mahalanobis_outliers_dscto_hora:
                clear_output()
                display(df_mahalanobis_dscto_hora[0])
            with out_abod:
                print('Espere por favor...')
                clear_output(True)
                fig7 = go.Figure()
                fig7.add_trace(go.Scatter(name = 'outliers', mode = 'markers', x = df_abod_monto_dscto[0]['monto'], y = df_abod_monto_dscto[0]['descuento'], marker = dict(color = color_alert)))
                fig7.add_trace(go.Scatter(name = 'inliers', mode = 'markers', x = df_abod_monto_dscto[1]['monto'], y = df_abod_monto_dscto[1]['descuento'], marker = dict(color = color_no_alert)))
                fig7.update_layout(title_text = 'Gráfico de dispersión de la variable "monto" vs. "descuento" del cliente {}'.format(cliente), autosize = False, width = 1000, height = 400)
                fig7.layout['yaxis1'].title = 'descuento (soles)'
                fig7.layout['xaxis1'].title = 'monto (soles)'
                fig7.show()

                fig8 = go.Figure()
                fig8.add_trace(go.Scatter(name = 'outliers', mode = 'markers', x = df_abod_monto_hora[0]['hora_en_segundos'], y = df_abod_monto_hora[0]['monto'], marker = dict(color = color_alert)))
                fig8.add_trace(go.Scatter(name = 'inliers', mode = 'markers', x = df_abod_monto_hora[1]['hora_en_segundos'], y = df_abod_monto_hora[1]['monto'], marker = dict(color = color_no_alert)))
                fig8.update_layout(title_text = 'Gráfico de dispersión de la variable "monto" vs. hora de "fecVenta" del cliente {}'.format(cliente), autosize = False, width = 1000, height = 400)
                fig8.layout['yaxis1'].title = 'monto (soles)'
                fig8.layout['xaxis1'].title = 'fecVenta (hora)'
                fig8.layout['xaxis1'].tickvals = segundos_boxplot_ticks
                fig8.layout['xaxis1'].ticktext = hora_boxplot_ticks
                fig8.show()

                fig9 = go.Figure()
                fig9.add_trace(go.Scatter(name = 'outliers', mode = 'markers', x = df_abod_dscto_hora[0]['hora_en_segundos'], y = df_abod_dscto_hora[0]['descuento'], marker = dict(color = color_alert)))
                fig9.add_trace(go.Scatter(name = 'inliers', mode = 'markers', x = df_abod_dscto_hora[1]['hora_en_segundos'], y = df_abod_dscto_hora[1]['descuento'], marker = dict(color = color_no_alert)))
                fig9.update_layout(title_text = 'Gráfico de dispersión de la variable "descuento" vs. hora de "fecVenta" del cliente {}'.format(cliente), autosize = False, width = 1000, height = 400)
                fig9.layout['yaxis1'].title = 'descuento (soles)'
                fig9.layout['xaxis1'].title = 'fecVenta (hora)'
                fig9.layout['xaxis1'].tickvals = segundos_boxplot_ticks
                fig9.layout['xaxis1'].ticktext = hora_boxplot_ticks
                fig9.show()
            with out_abod_outliers_monto_dscto:
                clear_output()
                display(df_abod_monto_dscto[0])
            with out_abod_outliers_monto_hora:
                clear_output()
                display(df_abod_monto_hora[0])
            with out_abod_outliers_dscto_hora:
                clear_output()
                display(df_abod_dscto_hora[0])
            display(tab_graficos_tab_4)

    btn_refrescar_graficos_outliers.on_click(on_click_btn_refrescar_graficos_outliers)
    box_controles_tab_4 = wd.VBox(children = [lbl_clientes_outliers, dpw_clientes_outliers, lbl_confianza_mahalanobis, box_flt_confianza_mahalanobis, lbl_sliders, sld_monto, sld_dscto, sld_hora, lbl_slider_hora, btn_refrescar_graficos_outliers])
    tab_children_4 = wd.VBox(children = [html_descripcion_tab_4, wd.HBox(children = [box_controles_tab_4, out_graficos_tab_4])])

    # Widgets del tab_children_5
    html_descripcion_tab_5 = wd.HTML('<h2>Etiquetado de outliers</h2><br>En esta sección puedes etiquetar a los registros detectados como outliers. Las opciones son "Ruido", "Anomalía" o "Normal" (inlier).')

    def obtener_outliers_consolidado(metodo):
        tupla_indices = None
        if metodo == 'Boxplot':
            tupla_indices = (dividir_dataset_outliers(df, ['monto'], 'Boxplot', False).index.values,
                             dividir_dataset_outliers(df, ['descuento'], 'Boxplot', False).index.values,
                             dividir_dataset_outliers(df, ['hora_en_segundos'], 'Boxplot', False).index.values)
        elif metodo == 'Mahalanobis':
            tupla_indices = (dividir_dataset_outliers(df, ['monto','descuento'], 'Mahalanobis', False).index.values,
                             dividir_dataset_outliers(df, ['monto','hora_en_segundos'], 'Mahalanobis', False).index.values,
                             dividir_dataset_outliers(df, ['descuento','hora_en_segundos'], 'Mahalanobis', False).index.values)
        elif metodo == 'ABOD':
            tupla_indices = (dividir_dataset_outliers(df, ['monto','descuento'], 'ABOD', False).index.values,
                             dividir_dataset_outliers(df, ['monto','hora_en_segundos'], 'ABOD', False).index.values,
                             dividir_dataset_outliers(df, ['descuento','hora_en_segundos'], 'ABOD', False).index.values)

        for i, array in enumerate(tupla_indices):
            if i == 0:
                indices_outliers_total = array.copy()
            else:
                indices_outliers_total = np.append(indices_outliers_total, array)
        
        recuento_indices_outliers = pd.DataFrame(indices_outliers_total).value_counts()
        indices_outliers_total = [e[0] for e in recuento_indices_outliers[recuento_indices_outliers > 2].index.values]
        return df.loc[indices_outliers_total]

    def obtener_widgets_etiquetado(df_outliers):
        widgets_etiquetado = [wd.Label('Índice')]
        for e in df_outliers.columns:
            widgets_etiquetado.append(wd.Label(e))
        widgets_etiquetado.append(wd.Label('Etiqueta'))
        for ix, row in df_outliers.iterrows():
            widgets_etiquetado.append(wd.Label(str(ix)))
            for elem in row:
                widgets_etiquetado.append(wd.Label(str(elem) if type(elem) != 'str' else elem))
            widgets_etiquetado.append(wd.Dropdown(options = ['Normal', 'Ruido', 'Anomalía']))
        return widgets_etiquetado

    out_mensaje_etiquetado = wd.Output(layout = wd.Layout(margin = '5px 0 5px 0'))
    out_tabla_etiquetado = wd.Output()
    
    btn_etiquetar_ouliers = wd.Button(description = 'Etiquetar')
    def on_click_btn_etiquetar_ouliers(b):
        global gbx_tabla_etiquetado_boxplot
        global gbx_tabla_etiquetado_mahalanobis
        global flag_tab_etiquetado_mostrado
        with out_mensaje_etiquetado:
            clear_output()
            print('Por favor espere unos minutos a que se rendericen los widgets...')
        df_outliers_boxplot = obtener_outliers_consolidado(metodo = 'Boxplot')
        df_outliers_mahalanobis = obtener_outliers_consolidado(metodo = 'Mahalanobis')
        #df_outliers_abod = obtener_outliers_consolidado(metodo = 'ABOD')
        tabla_etiq_boxplot = obtener_widgets_etiquetado(df_outliers_boxplot)
        tabla_etiq_mahalanobis = obtener_widgets_etiquetado(df_outliers_mahalanobis)
        #tabla_etiq_abod = obtener_widgets_etiquetado(df_outliers_abod)
        gbx_tabla_etiquetado_boxplot = wd.GridBox(tabla_etiq_boxplot, layout = wd.Layout(grid_template_columns = "repeat(9, 100px)"))
        gbx_tabla_etiquetado_mahalanobis = wd.GridBox(tabla_etiq_mahalanobis, layout = wd.Layout(grid_template_columns = "repeat(9, 100px)"))
        #gbx_tabla_etiquetado_abod = wd.GridBox(tabla_etiq_abod, layout = wd.Layout(grid_template_columns = "repeat(9, 100px)"))

        tab_etiquetado = wd.Tab(children = [gbx_tabla_etiquetado_boxplot
                                            ,gbx_tabla_etiquetado_mahalanobis
                                            #,gbx_tabla_etiquetado_abod
                                            ])
        tab_etiquetado.set_title(0, 'Boxplot')
        tab_etiquetado.set_title(1, 'Mahalanobis')
        #tab_etiquetado.set_title(2, 'ABOD')
        with out_mensaje_etiquetado:
            clear_output()
            print('Ya puede etiquetar')
        with out_tabla_etiquetado:
            clear_output()
            display(tab_etiquetado)
        flag_tab_etiquetado_mostrado = True
    btn_etiquetar_ouliers.on_click(on_click_btn_etiquetar_ouliers)

    btn_descargar_etiquetas = wd.Button(description = 'Descargar xlsx')
    def on_click_btn_descargar_etiquetas(b):
        if not flag_tab_etiquetado_mostrado:
            with out_mensaje_etiquetado:
                clear_output()
                print('Etiqueta primero los registros para poder descargar el archivo.')
        else:
            with out_mensaje_etiquetado:
                clear_output()
                print('Preparando archivo para la descarga...')
            indices_boxplot = []
            indices_mahalanobis = []
            etiquetas_boxplot = []
            etiquetas_mahalanobis = []
            flag_header = True
            for i in range(0, len(gbx_tabla_etiquetado_boxplot.children), 9):
                if flag_header:
                    flag_header = False
                    continue
                indices_boxplot.append(int(gbx_tabla_etiquetado_boxplot.children[i].value))
                etiquetas_boxplot.append(gbx_tabla_etiquetado_boxplot.children[i + 8].value)
            
            flag_header = True
            for i in range(0, len(gbx_tabla_etiquetado_mahalanobis.children), 9):
                if flag_header:
                    flag_header = False
                    continue
                indices_mahalanobis.append(int(gbx_tabla_etiquetado_mahalanobis.children[i].value))
                etiquetas_mahalanobis.append(gbx_tabla_etiquetado_mahalanobis.children[i + 8].value)

            df_descarga = df.copy()
            df_descarga = df_descarga.assign(etiqueta_outlier_boxplot = 'Normal')
            df_descarga = df_descarga.assign(etiqueta_outlier_mahalanobis = 'Normal')
            df_descarga.loc[indices_boxplot, 'etiqueta_outlier_boxplot'] = etiquetas_boxplot
            df_descarga.loc[indices_mahalanobis, 'etiqueta_outlier_mahalanobis'] = etiquetas_mahalanobis
            df_descarga.to_excel('PY01_etiquetado.xlsx')
            with out_mensaje_etiquetado:
                clear_output()
                print('Ha sido descargado el archivo PY01_etiquetado.xlsx')

    btn_descargar_etiquetas.on_click(on_click_btn_descargar_etiquetas)
    
    tab_children_5 = wd.VBox(children = [html_descripcion_tab_5, wd.HBox(children = [btn_etiquetar_ouliers, btn_descargar_etiquetas]), out_mensaje_etiquetado, out_tabla_etiquetado])

    # Widgets del tab_children_6
    html_descripcion_tab_6 = wd.HTML('<h2>Detección de eventos sistémicos</h2><br>En esta sección puedes revisar el comportamiento de las variables cuantitativas a través del tiempo (fecVenta). La línea punteada indica la media y el sombreado alrededor de ella la desviación estándar.')
    dpw_variable_sistemicos = wd.Dropdown(options = [('Monto','monto'), ('Descuento','descuento'), ('Hora (fecVenta)','hora_en_segundos')], value = None)
    out_graficos_sistemicos = wd.Output(layout = wd.Layout(margin = '5px 0 0 0'))

    df_sist = df.assign(solo_fecha = df.fecha.dt.date)
    df_sist = df_sist.groupby(by = ['solo_fecha'], axis = 0, as_index = False).agg({'monto':'sum', 'descuento':'sum', 'hora_en_segundos':'mean'})
    def handle_dpw_variable_sistemicos_update(change):
        media_variable = df_sist[change.new].mean()
        stdev_variable = df_sist[change.new].std()
        fig = go.Figure()
        fig.add_trace(go.Scatter(mode = 'markers+lines', x = df_sist['solo_fecha'], y = df_sist[change.new]))
        fig.add_hline(y = max(df_sist[change.new]), name = 'Máximo')
        fig.add_hline(y = media_variable, line_dash = 'dash', name = 'Media')
        fig.add_hline(y = min(df_sist[change.new]), name = 'Mínimo')
        fig.add_hrect(y0 = media_variable - stdev_variable, y1 = media_variable + stdev_variable, line_width = 0, fillcolor = "grey", opacity = 0.3)
        if change.new == 'hora_en_segundos':
            fig.layout['yaxis1'].tickvals = segundos_boxplot_ticks
            fig.layout['yaxis1'].ticktext = hora_boxplot_ticks
        with out_graficos_sistemicos:
            clear_output()
            fig.show()
    
    dpw_variable_sistemicos.observe(handle_dpw_variable_sistemicos_update, names = 'value')
    tab_children_6 = wd.VBox(children = [html_descripcion_tab_6, dpw_variable_sistemicos, out_graficos_sistemicos])

    # Widgets del tab_children_7
    html_descripcion_tab_7 = wd.HTML('<h2>Hipótesis</h2><br>En esta sección puedes revisar las distintas hipótesis planteadas y verificar si la data los respalda.')
    dpw_hipotesis = wd.Dropdown(options = [
                                           ('H1: En los días viernes se da un mayor monto de compra',1),
                                           ('H2: Mayor monto de compra durante la 2da y 4ta semana',2),
                                           ('H3: Hay diferencia en el monto de compra diaria entre ambos sexos',3),
                                           ('H4: Hay diferencia en el monto de compra entre ventas durante el horario de atención y fuera de',4),
                                           ('H5: Un cliente fidelizado tiene una frecuencia de al menos una vez por semana',5)], value = None)
    out_graficos_hipotesis = wd.Output(layout = wd.Layout(margin = '5px 0 0 0'))

    def handle_dpw_hipotesis_update(change):
        with out_graficos_hipotesis:
            clear_output()
            if change.new == 1:
                df_h1 = df.assign(dia_semana_fecha = df.fecha.map(lambda x: str(x.isoweekday())))
                fig = px.box(data_frame = df_h1, x = 'dia_semana_fecha', y = 'monto', points = 'all')
                fig.update_xaxes(title = 'Día de la semana', categoryorder = 'category ascending')
                fig.update_yaxes(title = 'Monto (en soles)')
                fig.show()
            elif change.new == 2:
                df_h2 = df.assign(semana_mes_fecha = df.fecha.map(lambda x: str(math.ceil(x.day / 7.0))))
                fig = px.box(data_frame = df_h2, x = 'semana_mes_fecha', y = 'monto', points = 'all')
                fig.update_xaxes(title = 'Semana del mes', categoryorder = 'category ascending')
                fig.update_yaxes(title = 'Monto (en soles)')
                fig.show()
            elif change.new == 3:
                df_h3 = df[(df['sexo'] == 'Femenino') | (df['sexo'] == 'Masculino')]
                fig = px.box(data_frame = df_h3, x = 'sexo', y = 'monto', points = 'all')
                fig.update_xaxes(title = 'Sexo del cliente', categoryorder = 'category ascending')
                fig.update_yaxes(title = 'Monto (en soles)')
                fig.show()
            elif change.new == 4:
                df_h4 = df.assign(flag_domingo_feriado_fecha = df.fecha.map(lambda x: False if x in (datetime(2021,4,1).date(),datetime(2021,4,2).date(),datetime(2022,4,14).date(),datetime(2022,4,15).date(),datetime(2023,4,6).date(),datetime(2023,4,7).date(),datetime(2024,3,28).date(),datetime(2024,3,29).date()) or (x.month == 1 and x.day == 1) or (x.month == 5 and x.day == 1) or (x.month == 6 and x.day == 29) or (x.month == 7 and x.day == 28) or (x.month == 7 and x.day == 29 and x.year != 2020) or (x.month == 8 and x.day == 30) or (x.month == 10 and x.day == 8) or (x.month == 11 and x.day == 1) or (x.month == 12 and x.day == 8) or (x.month == 12 and x.day == 25) or x.isoweekday() == 7 or (x.isoweekday() in range(1,7) and not (8 <= (x.hour + x.minute/60) <= 18)) else True))
                fig = px.box(data_frame = df_h4, x = 'flag_domingo_feriado_fecha', y = 'monto', points = 'all')
                fig.update_xaxes(title = 'Horario de atención', categoryorder = 'category ascending', tickvals = [True, False], ticktext = ['Durante', 'Fuera de horario'])
                fig.update_yaxes(title = 'Monto (en soles)')
                fig.show()
            elif change.new == 5:
                min_fecha = df.fecha.dt.date.min()
                max_fecha = df.fecha.dt.date.max()
                min_eow = min_fecha + timedelta(days = (7 - min_fecha.isoweekday()))
                max_eow = max_fecha + timedelta(days = (7 - max_fecha.isoweekday()))
                cantidad_semanas = int((max_eow - min_eow).days/7) + 1
                lista_eow = [min_eow + timedelta(days = 7 * semana) for semana in range(cantidad_semanas)]

                df_h5 = df.assign(solo_fecha = df['fecha'].dt.date)
                fechas_compra_cliente = df_h5[df_h5['cliente'] != 'SID1'][['cliente','solo_fecha']].groupby(by = ['cliente','solo_fecha'], as_index = False).size().values.tolist()
                fechas_compra_cliente = sorted(fechas_compra_cliente, key = lambda x: (int(x[0][3:]), x[1]))

                clientes = sorted([*{*(row[0] for row in fechas_compra_cliente)}], key = lambda x: int(x[3:]))

                res = []
                for cliente in clientes:
                    #frecuencias_por_semana = [0 for i in range(cantidad_semanas)]
                    #compras_por_semana = [0 for i in range(cantidad_semanas)]
                    frecuencia_total = 0
                    cantidad_semanas_compra = 0
                    for cliente_lista, fecha_lista, frecuencia_lista in fechas_compra_cliente:
                        if cliente == cliente_lista:
                            frecuencia_total = frecuencia_total + frecuencia_lista
                            for i, eow in enumerate(lista_eow):
                                if fecha_lista <= eow:
                                    #frecuencias_por_semana[i] = frecuencias_por_semana[i] + frecuencia_lista
                                    #compras_por_semana[i] = 1
                                    cantidad_semanas_compra += 1
                                    break
                    
                    #res.append((cliente, frecuencias_por_semana, frecuencia_total))
                    #res.append((cliente, compras_por_semana, cantidad_semanas_compra))
                    #res.append((cliente, cantidad_semanas_compra, frecuencia_total))
                    res.append(cantidad_semanas_compra)
                fig = px.histogram(res)
                fig.update_xaxes(title = 'Cantidad de semanas frecuentadas ({} semanas en total)'.format(cantidad_semanas), tickvals = [*range(2,cantidad_semanas + 1,2)], range=[0, cantidad_semanas])
                fig.update_yaxes(title = 'Frecuencia')
                fig.update_layout(showlegend = False)
                fig.show()

    dpw_hipotesis.observe(handle_dpw_hipotesis_update, names = 'value')
    tab_children_7 = wd.VBox(children = [html_descripcion_tab_7, dpw_hipotesis, out_graficos_hipotesis])

    # Widgets del tab_children_8
    html_descripcion_tab_8 = wd.HTML('<h2>Curvas de aprendizaje</h2><br>En esta sección puedes ver las curvas de aprendizaje para validar los clasificadores entrenados.')
    btn_validar_clasificadores = wd.Button(description = 'Mostrar', button_style = 'success', icon = 'check')
    out_validacion_clasificadores = wd.Output()

    def on_btn_validar_clasificadores_clicked(b):
        with out_validacion_clasificadores:
            clear_output()
            curvas = obtener_curvas_aprendizaje()
            fig, axes = plt.subplots(1, 3, figsize = (20, 5))
            i_axes = 0
            for cliente, data_curva in curvas.items():
                train_sizes = list()
                train_scores = list()
                test_scores = list()
                for (train_size, (train_score, test_score), (_, _, _)) in data_curva:
                    train_sizes.append(train_size)
                    train_scores.append(train_score)
                    test_scores.append(test_score)
                    
                if len(train_sizes) == 1: continue

                axes[i_axes].set_title('Curva de aprendizaje de {}'.format(cliente))
                axes[i_axes].set_xlabel('Cantidad de registros de entrenamiento')
                axes[i_axes].set_ylabel('F1 score')
                axes[i_axes].set_xticks(train_sizes)
                axes[i_axes].grid()
                axes[i_axes].plot(train_sizes, train_scores, 'o-', color = 'r', label = 'Puntaje de entrenamiento')
                axes[i_axes].plot(train_sizes, test_scores, 'o-', color = 'g', label = 'Puntaje de validación')
                axes[i_axes].legend(loc = 'best')

                i_axes += 1

            plt.show()
            
            clasificadores = obtener_scores_clasificadores()
            datos_tabla_comparativa = list()
            df_modelo = crear_dataframe_desde_mongodb()
            df_modelo = df_modelo[(df_modelo.anho == '2020') & (df_modelo.mes == 'Julio')][['cliente','cantidad']]
            for cliente, data_clasificador in clasificadores.items():
                _, _, scores, _, n_classes= data_clasificador
                if n_classes == 1: continue
                cantidad_registros = sum(pd.to_numeric(df_modelo[df_modelo.cliente == cliente].cantidad))
                datos_tabla_comparativa.append((cliente, scores['mean_test_f1_weighted'], scores['mean_test_accuracy'], n_classes, cantidad_registros))
            
            header = ['Cliente','Media puntaje F1 CV','Media Exactitud CV', 'Cantidad de clases (prod)', 'Cantidad de registros']
            tabla_comparativa = pd.DataFrame(datos_tabla_comparativa, columns = header)
            display(tabla_comparativa)
            
    btn_validar_clasificadores.on_click(on_btn_validar_clasificadores_clicked)

    tab_children_8 = wd.VBox(children = [html_descripcion_tab_8, btn_validar_clasificadores, out_validacion_clasificadores])

    # Tab
    tab = wd.Tab(children = [tab_children_1, tab_children_2, tab_children_3, tab_children_4, tab_children_5, tab_children_6, tab_children_7, tab_children_8])

    tab.set_title(0, 'Más relevantes')
    tab.set_title(1, 'Menos relevantes')
    tab.set_title(2, 'Evaluar registro')
    tab.set_title(3, 'Ver outliers')
    tab.set_title(4, 'Etiquetar outliers')
    tab.set_title(5, 'Eventos sistémicos')
    tab.set_title(6, 'Hipótesis')
    tab.set_title(7, 'Validar')

    display(tab)

lbl_fuente_datos = wd.Label('Escoja la fuente de datos:')
rd_fuente_datos = wd.RadioButtons(options = ['SQL', 'Archivo'], value = None)
out_fuente_datos = wd.Output()

btn_cargar_archivo = wd.FileUpload(accept = '.csv,.xlsx', multiple = False, icon = 'upload')
btn_procesar_archivo_cargado = wd.Button(description = 'Procesar datos', button_style = 'success', tooltip = 'Haz click para procesar los datos desde la fuente seleccionada', icon = 'check')

def handle_rd_fuente_datos_update(change):
    with out_fuente_datos:
        clear_output()
        if change.new == 'SQL':
            display(btn_procesar_archivo_cargado)
        elif change.new == 'Archivo':
            display(wd.HBox(children = [btn_cargar_archivo, btn_procesar_archivo_cargado]))
          
rd_fuente_datos.observe(handle_rd_fuente_datos_update, names = 'value')

def on_btn_procesar_archivo_cargado_clicked(b):
    flag_tab_etiquetado_mostrado = False
    if rd_fuente_datos.value == 'SQL':
        with out_mensaje_procesar_datos:
            clear_output()
            print('Espere un momento...')
        procesar_archivo(rd_fuente_datos.value)
        with out_cargar_archivo:
            clear_output()
            mostrar_widgets_visualizacion()
    elif rd_fuente_datos.value == 'Archivo':
        if btn_cargar_archivo.value:
            with out_mensaje_procesar_datos:
                clear_output()
                print('Espere un momento...')
            procesar_archivo(rd_fuente_datos.value)        
            with out_cargar_archivo:
                clear_output()
                mostrar_widgets_visualizacion()
        else:
            with out_mensaje_procesar_datos:
                clear_output()
                print('Cargue un archivo primero.')

btn_procesar_archivo_cargado.on_click(on_btn_procesar_archivo_cargado_clicked)
out_mensaje_procesar_datos = wd.Output(layout = wd.Layout(margin = '5px 0 5px 0'))
out_cargar_archivo = wd.Output()

html_desarrolladores = wd.HTML('<h3>Desarrolladores:</h3><ul><li>César Nieves</li><li>Christopher Fernández</li></ul>')
html_video_guia_uso = wd.HTML('<h3>Guía de uso:</h3><iframe width="800" height="420" src="https://www.youtube.com/embed/f_SG1Gqcg7c?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')
html_titulo_dashboard = wd.HTML('<h3>Dashboard:</h3>')
display(wd.VBox(children = [html_desarrolladores, html_video_guia_uso, html_titulo_dashboard, lbl_fuente_datos, rd_fuente_datos, out_fuente_datos, out_mensaje_procesar_datos, out_cargar_archivo]))

In [ ]:
while True:
    pass